In [ ]:
import os
import pandas as pd
pd.set_option('display.max_columns', 100, 'display.max_rows', 150)
import numpy as np
import matplotlib.pyplot as plt
import config

#Load google.cloud.bigquery
%load_ext google.cloud.bigquery

#Select path to credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=config.GOOGLE_APPLICATION_CREDENTIALS

In [ ]:
%%bigquery --use_rest_api ZRI_MF
SELECT *
FROM `high-empire-220313.ZRI.Multi_Family`

In [ ]:
ZRI_MF = ZRI_MF.drop(ZRI_MF[ZRI_MF['RegionName'] == 11901].index,axis = 0)

In [ ]:
ZRI_MF.to_pickle('./pickles/ZRI_MF_wide.p')

In [ ]:
def month_year(x):
    month = int(x[-2:])
    year = int(x[1:5])
    if year<2014:
        return(True)
    if (year == 2014) and month<3:
        return(True)
    return(False)

In [ ]:
time_columns = [x for x in ZRI_MF.columns if ('20' in x)]
time_drop = [x for x in time_columns if month_year(x)]

In [ ]:
ZRI_MF = ZRI_MF.drop(time_drop, axis = 1)

In [ ]:
ZRI_MF['Missing_Months'] = ZRI_MF.isna().apply(sum, axis = 1)

In [ ]:
def consecutive_missing(row):
    return(max(row.isnull().astype(int).groupby(row.notnull().astype(int).cumsum()).sum()))

In [ ]:
ZRI_MF.apply(consecutive_missing,axis = 1).value_counts().sort_index(ascending= False)

In [ ]:
ZRI_filtered = ZRI_MF[ZRI_MF.apply(consecutive_missing,axis = 1)<5]

In [ ]:
ZRI_filtered.loc[:,[x for x in ZRI_MF.columns if ('20' in x)]] = ZRI_filtered[[x for x in ZRI_MF.columns if ('20' in x)]].interpolate(method = 'linear',
                                                                     axis = 0)

In [ ]:
ZRI_filtered = ZRI_filtered.drop('Metro', axis = 1)

In [ ]:
ZRI_filtered.to_pickle('./pickles/ZRI_filtered.p')
# import pickle
# file = open('./pickles/ZRI_filtered.p','rb')
# filtered2 = pickle.load(file)